In [1]:
import pandas as pd
#import os
import csv
import country_converter as coco
import re
from pyproj import Transformer 


In [2]:

# Specify the filename
# File is in same directory as the code, so path of file doesn't need to be specified
filename = 'links.csv'
file_path = filename


print(file_path)


new_file = 'filtered_links.csv'
new_file_path = new_file


links.csv


## Create Filtered buses file

Create functions to get filtered file

In [3]:


def extract_coordinates(line_string):
    matches = re.findall(r'\(([^)]+)\)', line_string)
    if matches:
        # Split the first match by ',' to separate the source and destination coordinates
        src, dst = matches[0].split(',')
        
        # Split each coordinate by space to separate X and Y components and convert them to float
        src_x, src_y = tuple(map(float, src.strip().split(' ')))
        dst_x, dst_y = tuple(map(float, dst.strip().split(' ')))
        
        # Return the coordinates as tuples of floats
        return (src_x, src_y), (dst_x, dst_y)
    return (0.0, 0.0), (0.0, 0.0)  # Return zero coordinates if no matches are found

# Testing the function
src_coords, dst_coords = extract_coordinates('LINESTRING(22.939453 40.663973,23.192139 40.722283)')
print(f"Source Coordinates: {src_coords}")
print(f"Destination Coordinates: {dst_coords}")


def extract_tag_values(tags_string, tag_name):
    
    # tags are separated by a space
    tags_list = tags_string.split(" ")
    
    for tag in tags_list:
        tag = tag.strip("'")
        if "=>" in tag:
            tag_name_info, value = tag.split("=>")      #tag name and value separated by =>
        
            # goes through tags to see find specified tag name and info
            if tag_name_info.strip("'").strip() == tag_name:
                return value.strip("'").strip()         # additional .strip removes any extra spaces
        
    # Returns None if the specified tag name is not found
    return None



Source Coordinates: (22.939453, 40.663973)
Destination Coordinates: (23.192139, 40.722283)


In [4]:
with open(file_path, 'r', newline='') as csv_file, open(new_file_path, 'w', newline='') as new_csv_file:
    reader = csv.reader(csv_file)
    writer = csv.writer(new_csv_file)
    
    
    #Writing header to new file
    original_header = next(reader)
    geometry_index = original_header.index('geometry')
    new_header = original_header[:geometry_index] + ['src_coord', 'dst_coord', 'src_country', 'dst_country'] + original_header[geometry_index+1:]
    
    
    
    under_construction_index = new_header.index('under_construction')
    tags_index = new_header.index('tags')
    src_country_index = new_header.index('src_country')
    dst_country_index = new_header.index('dst_country')
    
    writer.writerow(new_header)
    
    
    for row in reader:
        under_construction_value = row[under_construction_index]
        
        # Rows where the buses are underconstruction are not included
        if under_construction_value != 't':
            
            
            ## Adjusting tags string
            # Combining the columns of tags in each row to be in one single column
            # Last two columns in rows contain details about coordinates, not tags
            last_tag_index = len(row) - 3
            
            tags_string = ' '.join(row[tags_index : last_tag_index])
            
            # manipulating the output of string
            tags_string = tags_string.replace('"', "'")
            if tags_string.startswith("''"):
                tags_string = tags_string[1:]
            if tags_string.endswith("''"):
                tags_string = tags_string[:-1]
                
            # Entire tags string is placed in single tags columns
            row[tags_index] = tags_string
            
            # Duplicate information is deleted
            row[tags_index + 1:-2] = []
            
            
            
            ## Adjusting the coordinates string
            # coordinates are split across two columns in a row
            combined_coords = f"{row[-2]},{row[-1]}"
            
            # Last two columns are replaced with the combined coordinates string
            row = row[:-2] + [combined_coords]
            
            
            # Extract source and destination coordinates from last columns in row
            src_coord, dst_coord = extract_coordinates(row[-1])
            
            # Last column in row that contained the string for coordinates is now replaced by 2 columns of source and destination coordinates
            row = row[:-1] + [src_coord, dst_coord]
            
            
            
            
            ## Extracting particular tag information
            
            # Access source and destination country and create new column
            src_country = extract_tag_values(tags_string, "country_1")
            row.append(src_country)
            
            dst_country = extract_tag_values(tags_string, "country_2")
            row.append(dst_country)
            
            if row[src_country_index] != 'NaN' or row[dst_country_index] != 'Nan':
                writer.writerow(row)
            
            #writer.writerow(row)

Check for any NaN values

In [5]:
df = pd.read_csv(new_file_path)
print(df.shape)
## 8889 14

nan_count1 = df['src_country'].isna().sum()
nan_count2 = df['dst_country'].isna().sum()

country_code_counts1 = df['src_country'].value_counts(dropna=False)
country_code_counts2 = df['dst_country'].value_counts(dropna=False)



print("\nNumber of NaN values in 'src_country' column:", nan_count1)
print("\nNumber of NaN values in 'dst_country' column:", nan_count2)

print("Number of occurrences of each country code1:")
print(country_code_counts1)
print("Number of occurrences of each country code2:")
print(country_code_counts2)


voltage_counts = df['voltage'].value_counts()
print("\nVoltage Counts: ", voltage_counts)

#nan_count 3


# Create a set of unique country codes from both 'src_country' and 'dst_country' columns excluding NaN
unique_country_codes = set(df['src_country'].dropna()).union(set(df['dst_country'].dropna()))

# Initialize a dictionary to store conversion results
conversion_results = {}

# Iterate over each unique country code
for code in unique_country_codes:
    # Convert the country code to the corresponding country name
    country_name = coco.convert(names=code, to='name_short')
    
    # Store the conversion result in the dictionary
    conversion_results[code] = country_name
    #conversion_results.sort_values

# Print the conversion results
for code, name in conversion_results.items():
    print(f"{code}: {name}")

(8889, 14)

Number of NaN values in 'src_country' column: 1345

Number of NaN values in 'dst_country' column: 1407
Number of occurrences of each country code1:
src_country
NaN    1345
FR      905
RU      859
ES      689
DE      545
IT      369
GB      320
UA      229
SE      220
PL      204
EG      196
NO      188
DK      187
LY      185
MA      180
CH      176
TR      166
SY      151
DZ      149
RO      129
PT      119
TN      102
CZ       95
FI       92
BY       76
BG       72
AT       70
BA       67
JO       63
RS       60
BE       58
SK       54
HU       52
IE       52
IS       39
HR       36
NL       35
GR       33
LT       29
IL       28
AL       28
KZ       28
MD       26
SA       25
EE       24
LB       19
IQ       18
LV       17
CY       13
LU       11
ME       11
MK       11
SI       11
NI        9
GE        8
IR        3
AZ        2
AM        1
Name: count, dtype: int64
Number of occurrences of each country code2:
dst_country
NaN    1407
FR      910
RU      880
ES      670
D

# Create Country Specified dataframes and csv files

In [6]:
# Function to create country csv files


def get_country_network_data(file_path, country_id, voltages):
    # Load the DataFrame
    EU_df = pd.read_csv(file_path)

    # Filter based on country and voltage
    filtered_df = EU_df[((EU_df['src_country'] == country_id) | (EU_df['dst_country'] == country_id)) & EU_df['voltage'].isin(voltages)].copy()
    filtered_df.sort_values(by = 'voltage' , ascending = True, inplace = True)
    filtered_df = filtered_df.dropna(subset=['voltage'])

    # Function to decide which bus ID to keep
    def decide_bus_id(existing_id, new_id):
        # Keeps the smaller bus ID
        return min(existing_id, new_id)

    # Dealing with duplicate nodes of the same coordinates but different bus IDs
    coord_to_bus_id = {}
    for index, row in filtered_df.iterrows():
        for coord_type in ['src_coord', 'dst_coord']:
            coord = row[coord_type]
            bus_id = row[coord_type.split('_')[0] + '_bus_id']
            
            if coord not in coord_to_bus_id:
                coord_to_bus_id[coord] = bus_id
            else:
                coord_to_bus_id[coord] = decide_bus_id(coord_to_bus_id[coord], bus_id)

    # Update the DataFrame using the mappings
    filtered_df.loc[:, 'src_bus_id'] = filtered_df['src_coord'].map(coord_to_bus_id)
    filtered_df.loc[:, 'dst_bus_id'] = filtered_df['dst_coord'].map(coord_to_bus_id)

    # Coordinate transformation
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    def transform_coord(coord):
        if isinstance(coord, str):
            coord = coord.strip("()").split(",")
            coord = tuple(map(float, coord))
        return transformer.transform(*coord)

    # Apply the coordinate transformation
    filtered_df[['src_x', 'src_y']] = filtered_df['src_coord'].apply(transform_coord).apply(pd.Series)
    filtered_df[['dst_x', 'dst_y']] = filtered_df['dst_coord'].apply(transform_coord).apply(pd.Series)

    # Save the filtered DataFrame to a CSV file named after the specified country
    output_file_path = f"{country_id}.csv"
    filtered_df.to_csv(output_file_path, index=False)
    print(f"File saved: {output_file_path}")

""" # Example usage of the function

new_file_path = 'path_to_your_data_file.csv'
process_electric_network_data(new_file_path, 'IE', [220, 380]) """


EU_filtered_data = new_file_path
get_country_network_data(EU_filtered_data, 'IE', [220, 380])

File saved: IE.csv


## Ireland Dataframe for 220kV and 380kV

In [7]:
EU_filtered_data = new_file_path
get_country_network_data(EU_filtered_data, 'IE', [220, 380])

File saved: IE.csv


## Luxemberg Dataframe 220kV - 14 nodes

In [8]:
europe_df = pd.read_csv('filtered_links.csv')

Lux_df = europe_df[((europe_df['src_country'] == 'LU') | (europe_df['dst_country'] == 'LU'))].copy()
print(Lux_df.shape)
display(Lux_df)

get_country_network_data(EU_filtered_data, 'LU', [220])

(17, 14)


,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
2629,4825,3078,3082,220.0,1,f,f,f,15234.671583,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(5.976562, 49.482401)","(6.16333000000001, 49.560852)",LU,LU
2630,4823,3080,3082,220.0,2,f,f,f,12817.362232,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.30615199999999, 49.632062)","(6.16333000000001, 49.560852)",LU,LU
2631,8594,3081,3082,220.0,1,f,f,f,9086.723072,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.03149400000001, 49.539469)","(6.16333000000001, 49.560852)",LU,LU
2632,4826,3082,3083,220.0,2,f,f,f,9189.423339,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.16333000000001, 49.560852)","(6.17431599999999, 49.646291)",LU,LU
2633,4830,3079,3083,220.0,2,f,f,f,9278.115069,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.26220699999999, 49.710273)","(6.17431599999999, 49.646291)",LU,LU
2634,4384,2646,3084,220.0,2,f,f,f,34983.312597,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.52587899999999, 49.965356)","(6.053467, 49.993615)",DE,LU
2635,4544,2645,3084,220.0,2,f,f,f,14289.423953,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.26220699999999, 49.993615)","(6.053467, 49.993615)",DE,LU
2636,8402,3079,3085,220.0,2,f,f,f,12997.968352,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.26220699999999, 49.710273)","(6.108398, 49.781264)",LU,LU
2637,4832,2645,3085,220.0,2,f,f,f,26354.180634,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.26220699999999, 49.993615)","(6.108398, 49.781264)",DE,LU
2638,4827,3085,3086,220.0,1,f,f,f,12669.417784,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(6.108398, 49.781264)","(6.17431599999999, 49.887557)",LU,LU


File saved: LU.csv


## Montenengro Dataframe 220kV 380kV - 12 nodes

In [9]:
europe_df = pd.read_csv('filtered_links.csv')

Mon_df = europe_df[((europe_df['src_country'] == 'ME') | (europe_df['dst_country'] == 'ME'))].copy()
print(Mon_df.shape)
display(Mon_df)

get_country_network_data(EU_filtered_data, 'ME', [220, 380])

(18, 14)


,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
4849,6563,5398,5399,220.0,1,f,f,f,34688.528377,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(19.27002, 42.366662)","(18.863525, 42.4802)",ME,ME
4850,6562,4392,5399,220.0,1,f,f,f,47993.873154,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(18.391113, 42.70666)","(18.863525, 42.4802)",BA,ME
4851,6595,4393,5400,380.0,1,f,f,f,81614.600483,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(18.391113, 42.70666)","(19.324951, 42.415346)",BA,ME
4852,6564,5398,5401,220.0,1,f,f,f,72072.418909,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(19.27002, 42.366662)","(19.56665, 42.892064)",ME,ME
4855,6594,5400,5406,380.0,1,f,f,f,76627.991661,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(19.324951, 42.415346)","(19.885254, 42.988576)",ME,ME
4917,6598,5406,5448,380.0,1,f,f,f,40489.633829,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(19.885254, 42.988576)","(20.214844, 42.698586)",ME,RS
4919,8097,4418,5449,132.0,1,f,f,f,29149.481013,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(18.325195, 42.90816)","(18.687744, 42.932296)",BA,ME
4920,6557,4412,5450,132.0,1,f,f,f,14748.197529,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(18.435059, 42.617791)","(18.533936, 42.496403)",BA,ME
5200,6558,5451,5807,132.0,1,f,f,f,16811.255520,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(19.423828, 43.500752)","(19.259033, 43.405047)",ME,RS
5402,8700,5398,5982,220.0,1,f,f,f,33503.716789,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(19.27002, 42.366662)","(19.500732, 42.163403)",AL,ME


File saved: ME.csv


## Iceland Dataframe 132kV 220kV

In [10]:
#europe_df = pd.read_csv('filtered_links.csv')

Iceland_df = europe_df[((europe_df['src_country'] == 'IS') | (europe_df['dst_country'] == 'IS'))].copy()
print(Iceland_df.shape)
display(Iceland_df)

get_country_network_data(EU_filtered_data, 'IS', [132, 220])

(39, 14)


,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
5511,9014,6093,6094,132.0,1,f,f,f,85606.056856,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-23.148193, 65.789264)","(-21.851807, 65.467386)",IS,IS
5512,9004,6092,6094,132.0,1,f,f,f,50348.096659,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-21.643066, 65.141497)","(-21.851807, 65.467386)",IS,IS
5513,8999,6095,6096,132.0,1,f,f,f,64281.827847,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-20.368652, 65.512963)","(-21.137695, 65.07676)",IS,IS
5514,9003,6092,6096,132.0,1,f,f,f,26321.650617,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-21.643066, 65.141497)","(-21.137695, 65.07676)",IS,IS
5515,9007,6089,6097,132.0,1,f,f,f,92467.258898,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-18.182373, 65.68543)","(-19.489746, 65.544819)",IS,IS
5516,9665,6098,6099,132.0,1,f,f,f,36216.000074,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-14.688721, 65.030423)","(-14.523926, 65.348514)",IS,IS
5517,9010,6098,6100,132.0,1,f,f,f,45224.851524,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-14.688721, 65.030423)","(-14.370117, 64.69441)",IS,IS
5518,9011,6100,6101,132.0,1,f,f,f,60292.067427,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-14.370117, 64.69441)","(-15.007324, 64.344665)",IS,IS
5519,8998,6101,6102,132.0,1,f,f,f,175701.329128,'text_'=>' ' 'symbol'=>'132-150 kV Transmissi...,"(-15.007324, 64.344665)","(-18.083496, 63.81644)",IS,IS
5520,8997,6104,6106,220.0,2,f,f,f,10373.355547,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(-21.719971, 64.086607)","(-21.851807, 64.014496)",IS,IS


File saved: IS.csv


## Greece Dataframe 380kV

In [19]:
#europe_df = pd.read_csv('filtered_links.csv')

Greece_df = europe_df[((europe_df['src_country'] == 'GR') | (europe_df['dst_country'] == 'GR'))].copy()
print(Greece_df.shape)
display(Greece_df)


get_country_network_data(EU_filtered_data, 'GR', [380])

(36, 14)


,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
55,577,101,103,380.0,1,f,f,f,78531.425893,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(21.137695, 39.27479)","(21.588135, 38.659778)",GR,GR
56,591,105,106,380.0,2,f,f,f,47349.953190,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(23.192139, 38.462192)","(22.70874, 38.401949)",GR,GR
57,586,103,106,380.0,2,f,f,f,99952.258136,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(21.588135, 38.659778)","(22.70874, 38.401949)",GR,GR
58,603,106,107,380.0,2,f,f,f,139519.067613,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(22.70874, 38.401949)","(21.994629, 39.487085)",GR,GR
59,576,101,107,380.0,1,f,f,f,82848.969383,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(21.137695, 39.27479)","(21.994629, 39.487085)",GR,GR
60,606,105,108,380.0,2,f,f,f,153617.468959,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(23.192139, 38.462192)","(22.434082, 39.529467)",GR,GR
61,605,105,108,380.0,2,f,f,f,148665.836773,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(23.192139, 38.462192)","(22.434082, 39.529467)",GR,GR
62,582,110,111,380.0,2,f,f,f,28568.375584,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(23.631592, 37.996163)","(23.741455, 38.22092)",GR,GR
63,2325,105,111,380.0,2,f,f,f,54046.793774,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(23.192139, 38.462192)","(23.741455, 38.22092)",GR,GR
64,589,109,112,380.0,2,f,f,f,15181.712338,'text_'=>'150' 'symbol'=>'380-400 kV Transmis...,"(23.796387, 37.857507)","(23.950195, 37.926868)",GR,GR


File saved: GR.csv


## Austria Dataframe, 220kV and 380kV

In [12]:
Austria_df = europe_df[((europe_df['src_country'] == 'AT') | (europe_df['dst_country'] == 'AT'))].copy()
print(Austria_df.shape)
print(Austria_df.to_string())
display(Austria_df)

get_country_network_data(EU_filtered_data, 'AT', [132, 220, 380])

(75, 14)
      link_id  src_bus_id  dst_bus_id  voltage  circuits dc underground under_construction       length_m                                                                                                                                                                                                          tags                      src_coord                             dst_coord src_country dst_country
3802     8194        4285        4292    220.0         2  f           f                  f   24597.151193                  'text_'=>' '  'symbol'=>'220 kV Transmission Line  2 Circuits'  'ogc_fid'=>'6868'  't9_code'=>'0'  'visible'=>'1'  'objectid'=>'7584'  'tie_line'=>'0'  'country_1'=>'AT'  'country_2'=>'AT'         (10.030518, 46.980252)         (9.78881800000001, 47.137425)          AT          AT
3803     8240        4291        4292    220.0         2  f           f                  f    6746.141861                  'text_'=>' '  'symbol'=>'220 kV Transmission Line  2 Circu

,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
3802,8194,4285,4292,220.0,2,f,f,f,24597.151193,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(10.030518, 46.980252)","(9.78881800000001, 47.137425)",AT,AT
3803,8240,4291,4292,220.0,2,f,f,f,6746.141861,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(9.7229, 47.092566)","(9.78881800000001, 47.137425)",AT,AT
3804,8234,4293,4295,380.0,2,f,f,f,20212.046764,'text_'=>'220' 'symbol'=>'380-400 kV Transmis...,"(9.78881800000001, 47.137425)","(9.62402299999999, 47.286682)",AT,AT
3805,8810,4294,4295,380.0,2,f,f,f,16131.017514,'text_'=>' 220' 'symbol'=>'380-400 kV Trans...,"(9.70092800000001, 47.428087)","(9.62402299999999, 47.286682)",AT,AT
3806,8216,4298,4299,220.0,2,f,f,f,12586.593138,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(10.997314, 47.144897)","(10.898438, 47.23449)",AT,AT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,8237,4317,7805,380.0,2,f,f,f,68153.044101,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(12.689209, 46.800059)","(12.601318, 47.316483)",AT,AT
8599,8349,4314,7805,380.0,2,f,f,f,5426.411178,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(12.667236, 47.346267)","(12.601318, 47.316483)",AT,AT
8684,8241,4310,7838,220.0,2,f,f,f,7798.749098,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(13.985596, 46.619261)","(13.985596, 46.54375)",AT,AT
8685,8346,7751,7838,220.0,2,f,f,f,14537.054921,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(14.18335, 46.551305)","(13.985596, 46.54375)",AT,AT


File saved: AT.csv


## Other Test Dataframes

In [13]:
Finland_df = europe_df[((europe_df['src_country'] == 'FI') | (europe_df['dst_country'] == 'FI'))].copy()
print(Finland_df.shape)
print(Finland_df.to_string())
display(Finland_df)

get_country_network_data(EU_filtered_data, 'FI', [220, 380])

(94, 14)
      link_id  src_bus_id  dst_bus_id  voltage  circuits dc underground under_construction       length_m                                                                                                                                                                                                      tags                             src_coord                             dst_coord src_country dst_country
2242     4206        2564        2565    380.0         1  f           f                  f   14109.679878           'text_'=>' '  'symbol'=>'380-400 kV Transmission Line  1 Circuit'  'ogc_fid'=>'1415'  't9_code'=>'0'  'visible'=>'1'  'objectid'=>'1510'  'tie_line'=>'0'  'country_1'=>'FI'  'country_2'=>'FI'                (24.609375, 65.879215)                (24.411621, 65.780251)          FI          FI
2243     4155        2561        2566    220.0         1  f           f                  f   21346.263619               'text_'=>' '  'symbol'=>'220 kV Transmission Line  1 Ci

,link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,src_coord,dst_coord,src_country,dst_country
2242,4206,2564,2565,380.0,1,f,f,f,14109.679878,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(24.609375, 65.879215)","(24.411621, 65.780251)",FI,FI
2243,4155,2561,2566,220.0,1,f,f,f,21346.263619,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(27.399902, 66.451887)","(27.026367, 66.328685)",FI,FI
2244,4230,2559,2566,220.0,1,f,f,f,12926.074971,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(26.773682, 66.377158)","(27.026367, 66.328685)",FI,FI
2245,4161,2568,2569,220.0,1,f,f,f,83960.040215,'text_'=>' ' 'symbol'=>'220 kV Transmission L...,"(28.45459, 64.58147)","(26.8396, 64.557881)",FI,FI
2246,4224,2567,2572,380.0,1,f,f,f,162175.952013,'text_'=>' ' 'symbol'=>'380-400 kV Transmissi...,"(27.026367, 66.328685)","(25.806885, 64.970064)",FI,FI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8846,4228,2558,2614,NaN,1,t,f,f,119827.661322,'text_'=>'Estlink 2' 'symbol'=>'DC-Line 1 Ci...,"(26.477051, 59.484148)","(25.64209, 60.440963)",EE,FI
8853,4152,2586,3826,NaN,1,t,f,f,207613.372205,'text_'=>'Fenno-Skan 1 400 kV' 'symbol'=>'DC-...,"(21.818848, 60.914414)","(18.017578, 60.532968)",FI,SE
8855,4217,2586,3856,NaN,1,t,f,f,262936.866721,'text_'=>'Fenno-Skan 2 (500kV)' 'symbol'=>'DC...,"(21.818848, 60.914414)","(17.061768, 60.549177)",FI,SE
8859,4216,2612,3872,NaN,1,t,f,f,80388.336030,'text_'=>'150 k (Estlink)' 'symbol'=>'DC-Line...,"(24.65332, 60.168842)","(24.389648, 59.43949)",EE,FI


File saved: FI.csv


In [16]:
Germany_df = europe_df[((europe_df['src_country'] == 'DE') | (europe_df['dst_country'] == 'DE'))].copy()
print(Germany_df.shape)
print(Germany_df.to_string())

(558, 14)
      link_id  src_bus_id  dst_bus_id  voltage  circuits dc underground under_construction       length_m                                                                                                                                                                                                                           tags                      src_coord                             dst_coord src_country dst_country
507      8903        3044        5821    132.0         1  f           f                  f   25366.986749                                                                                    'symbol'=>'132-150 kV Transmission Line  1 Circuit'  'ogc_fid'=>'10466'  't9_code'=>'0'  'visible'=>'1'  'tie_line'=>'0'  'country_1'=>'DE'          (12.546387, 54.57843)                (12.875977, 54.718275)          DE         NaN
2300     4436        2632        2633    220.0         2  f           f                  f   22196.362308                                   'text_